In [5]:
%pip install llama-index
%pip install transformers
%pip install torch
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-huggingface
%pip install accelerate
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import llama_index
import transformers
import torch


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    Settings,
)

/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInf

In [17]:
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

In [18]:
documents = SimpleDirectoryReader(input_files=["aapl-20220924.html"]).load_data()
index = VectorStoreIndex.from_documents(documents)

In [9]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [14]:
from llama_index.core import PromptTemplate


# Transform a string into Qwen-specific input (modify as needed for Qwen formatting)
def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


# Transform a list of chat messages into Qwen-specific input (adjust as needed)
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # Ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # Add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt


import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

# Update Settings for Qwen model
Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2-0.5B",  # Model name
    tokenizer_name="Qwen/Qwen2-0.5B",  # Tokenizer name
    context_window=2048,  # Adjust to match Qwen's context window size
    max_new_tokens=256,  # You can adjust this if needed
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},  # Generation parameters
    messages_to_prompt=messages_to_prompt,  # Custom prompt conversion for chat
    completion_to_prompt=completion_to_prompt,  # Custom completion conversion
    device_map="auto",  # Automatically assigns the model to available devices (e.g., GPU)
)


/opt/anaconda3/lib/python3.12/site-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 671089920 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the disk.


In [20]:
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [21]:
query_engine = index.as_query_engine(embedding_model=embedding_model)
response = query_engine.query("How was the company's performance over the period of the document?")
print(response)

/opt/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


The company's performance over the period of the document is not provided in the given context information.
